In [1]:
import datetime
def info(str_):
    print(f'{datetime.datetime.now()} [ INFO  ] {str_}')
def ok(str_):
    print(f'{datetime.datetime.now()} [ OK    ] {str_}')
def warning(str_):
    print(f'{datetime.datetime.now()} [WARNING] {str_}')
info('Starting Script...')

2022-01-31 16:28:34.326630 [ INFO  ] Starting Script...


In [2]:
#imports
import os
import spacy
import pandas as pd
import pickle
import numpy as np

nlp = spacy.load('en_core_web_lg', disable=['textcat','parser','tagger','ner','lemmatizer'])


In [3]:
from lxml import etree

def get_text(file):
    tree = etree.parse(file)
    root = tree.getroot()
    if root.find('.//HiddenText') is not None:
        return(root.find('.//HiddenText').text)
    
    elif root.find('.//Text') is not None:
        return(root.find('.//Text').text)
    
    else:
        return None
def get_title(file):
    tree = etree.parse(file)
    root = tree.getroot()
    return root.find('.//Title').text


In [14]:
from tqdm import tqdm

#to-do
GM_datapath = '/home/ec2-user/SageMaker/data/The_Globe_and_Mail_with_DP_filter_by_article_type/'
TS_datapath = '/home/ec2-user/SageMaker/data/Toronto_Star_Publication_with_query/'
vectors_datapath = '/home/ec2-user/SageMaker/mariano/notebooks/03. High Recall Retrieval System/vectors/'
id2vector = pickle.load(open(vectors_datapath+'id2vec.p', 'rb'))

batch_size=10
# def setup_model(batch_size=10):
info('Setting up')
info('Loading data...')
#to-do
GM_filenames = os.listdir(GM_datapath)
TS_filenames = os.listdir(TS_datapath)
info(f"Number of articles from The Globe and Mail: {len(GM_filenames):,}")
info(f"Number of articles from Toronto Star: {len(TS_filenames):,}")
info(f"Number of articles (total): {len(TS_filenames)+len(GM_filenames):,}")
ids_ = np.array([int(elem[:-4]) for elem in TS_filenames+GM_filenames]).reshape(-1,1)
data = pd.DataFrame(
    np.hstack([ids_, np.zeros(shape=(ids_.shape[0],300),dtype='float32')]),
    columns = ['ID']+[f'X{i}' for i in range (1,301)]
)
data['Source'] = ['TS']*len(TS_filenames) + ['GM']*len(GM_filenames)
data['Label'] = 'UNK'
info(f'Data shape: {data.shape}')
info('Loading vectors...')
vectors_list = []
for idx,id_ in enumerate(data['ID']):
    assert id_ in id2vector
    vectors_list.append(id2vector[id_])

data.iloc[:,1:301]= np.vstack(vectors_list)

# print(data.head())
ok('Vectors Lodaded.')



2022-01-19 20:05:29.866342 [ INFO  ] Setting up
2022-01-19 20:05:29.866507 [ INFO  ] Loading data...
2022-01-19 20:05:29.990258 [ INFO  ] Number of articles from The Globe and Mail: 151,004
2022-01-19 20:05:29.990432 [ INFO  ] Number of articles from Toronto Star: 41,429
2022-01-19 20:05:29.990927 [ INFO  ] Number of articles (total): 192,433
2022-01-19 20:05:30.386653 [ INFO  ] Data shape: (192433, 303)
2022-01-19 20:05:30.386804 [ INFO  ] Loading vectors...
2022-01-19 20:06:26.666151 [ OK    ] Vectors Lodaded.


In [13]:
from whoosh.qparser import MultifieldParser
from whoosh import index

index_dirpath = '/home/ec2-user/SageMaker/mariano/notebooks/03. High Recall Retrieval System/index'

# Loading Index
if index.exists_in(index_dirpath):
    print('Loading index')
    ix = index.open_dir(index_dirpath)
    
info(f'Number of documents in index: {ix.doc_count():,}')

mp = MultifieldParser(['title','body'],schema=ix.schema)
q = mp.parse(u'Refugee')

with ix.searcher() as s:
    results = s.search(q)
    
    print(results[0])
    hit = results[0]

Loading index
2022-01-20 19:35:11.484241 [ INFO  ] Number of documents in index: 192,427
<Hit {'file_id': '1140932582.xml', 'source': 'The Globe and Mail'}>


In [20]:
results[1]

ReaderClosed: 

In [18]:
query = input('Please enter search terms: ')
query_vector = nlp(query).vector


print('Retrieving and sorting relevant news articles...')
dist = np.linalg.norm(query_vector-data.iloc[:,1:301],axis=1)
#     print(dist)
#     print(dist.shape)
print()
info("Showing 5 most similar docs:")
# print(data.iloc[np.argsort(dist),:].shape)
# print(data.iloc[np.argsort(dist),:])
for item in data.iloc[np.argsort(dist),:].values[:5,:]:
    id_ = int(item[0])
    source = (item[301])
    filename = TS_datapath+str(id_)+'.xml' if source=='TS' else GM_datapath+str(id_)+'.xml'
#     print(int(id_))
#     print(str(item))
    print(f'{int(id_)} {source} {get_title(filename)}')
info("Showing 5 least similar docs:")
for item in data.iloc[np.argsort(dist),:].values[-5:,:]:
#     item.shape
#     break
    id_ = int(item[0])
    source = (item[301])
    filename = TS_datapath+str(id_)+'.xml' if source=='TS' else GM_datapath+str(id_)+'.xml'
    print(f'{int(id_)} {source} {get_title(filename)}')
#     print(str(item))
#to-do

print(f'Please label the following {batch_size} news articles...')
#to-do

print('Training computational model (classifier) to detect relevant news...')
#to-do
    

Please enter search terms: ireland refugees
Retrieving and sorting relevant news articles...

2022-01-19 20:35:04.129056 [ INFO  ] Showing 5 most similar docs:
1142660531 GM Other 11 -- No Title
1282764639 GM 3 Refugees Flee
1769885502 TS Syrian refugees by the numbers
1800388096 TS Syrian refugees by the numbers
1354244133 GM Russian Refugees For Canada
2022-01-19 20:35:26.016336 [ INFO  ] Showing 5 least similar docs:
1316932844 GM THE REVOLUTIONARY PRESS IN CHINA
1316932995 GM COUNTRY GIRLS
1316933019 GM Display Ad 32 -- No Title
1356533244 GM Other 19 -- No Title
1242297681 GM Other 25 -- No Title
Please label the following 10 news articles...
Training computational model (classifier) to detect relevant news...


In [ ]:
data.shape

In [13]:
data.iloc[np.argsort(dist),:]

,ID,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X293,X294,X295,X296,X297,X298,X299,X300,Source,Label
12875,4.396028e+08,0.001610,0.191167,-0.152071,-0.074249,0.085794,-0.022423,0.012251,-0.108927,0.011258,...,-0.003086,-0.039762,0.141711,-0.016706,-0.039697,-0.025963,0.009914,0.100015,TS,UNK
53336,1.371112e+09,-0.077209,0.142812,-0.034959,-0.001959,0.091572,0.011593,0.040312,-0.009059,0.004806,...,-0.037874,-0.082466,0.034255,-0.080199,-0.037502,-0.047243,0.035256,0.022693,GM,UNK
156085,1.353703e+09,-0.055947,0.097643,-0.176784,0.008479,0.115848,-0.036872,0.009258,-0.007494,-0.021282,...,-0.102679,-0.073735,0.143336,-0.087418,0.001705,0.035070,0.023013,0.009749,GM,UNK
43249,1.147309e+09,-0.000751,0.137542,0.009236,-0.065569,0.067999,-0.005468,0.040771,-0.070754,0.027744,...,-0.027044,-0.035335,0.070660,-0.059595,0.025985,-0.082864,-0.065280,0.011437,GM,UNK
175495,1.149767e+09,0.009771,0.135932,0.011016,-0.063189,0.083428,-0.020240,0.042791,-0.070850,0.025378,...,-0.019260,-0.045226,0.068942,-0.061062,0.017723,-0.078768,-0.049855,0.011774,GM,UNK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42901,1.316933e+09,-0.152883,-0.014833,0.044463,-0.032302,0.044214,0.348853,0.060872,-0.121309,-0.001245,...,0.102488,0.116578,0.147016,-0.132307,-0.246477,0.157300,-0.299183,0.042120,GM,UNK
103830,1.316933e+09,-0.152883,-0.014833,0.044463,-0.032302,0.044214,0.348853,0.060872,-0.121309,-0.001245,...,0.102488,0.116578,0.147016,-0.132307,-0.246477,0.157300,-0.299183,0.042120,GM,UNK
102974,1.316933e+09,-0.152883,-0.014833,0.044463,-0.032302,0.044214,0.348853,0.060872,-0.121309,-0.001245,...,0.102488,0.116578,0.147016,-0.132307,-0.246477,0.157300,-0.299183,0.042120,GM,UNK
154574,1.316933e+09,-0.152883,-0.014833,0.044463,-0.032302,0.044214,0.348853,0.060872,-0.121309,-0.001245,...,0.102488,0.116578,0.147016,-0.132307,-0.246477,0.157300,-0.299183,0.042120,GM,UNK


In [ ]:
count=0
for item in data.iloc[np.argsort(dist),:].values:
    print(item)
    count+=1
    if count==10:
        break

In [ ]:
    
def update_model():
    print('Running one loop iteration')
    
    print('Sorting news articles, picking most informatives to label...')
    #to-do
    
    print(f'Please label the following {batch_size} news articles...')
    #to-do
    
def model_status():
    print('Status')
setup_model()

In [4]:
GM_filenames = os.listdir(globe_and_mail_datapath)
TS_filenames = os.listdir(toronto_star_datapath)
print(f"Number of articles from The Globe and Mail: {len(GM_filenames):,}")
print(f"Number of articles from Toronto Star: {len(TS_filenames):,}")
ids_ = np.array([int(elem[:-4]) for elem in TS_filenames+GM_filenames])
ids_=ids_.reshape(-1,1)
ids_.append( np.zeros(shape=(192433,300)))

NameError: name 'os' is not defined

In [29]:
ids_.shape

(192433, 1)

In [31]:
m = np.zeros(shape=(192433,300))
m.shape

(192433, 300)

In [34]:
np.hstack([ids_,m]).shape

(192433, 301)

# set up model, run only once

In [ ]:
setup_model()

# update model (run as many time as needed)


In [ ]:
update_model()

# print model status (run in-between updates)

In [ ]:
model_status()

In [ ]:
import pigeonXT

In [4]:
import pandas as pd
import pigeonXT as pixt

for i in range(1):
    annotations = pixt.annotate(
        ['I love this movie', 'I was really disappointed by the book'],
        options=['positive', 'negative', 'inbetween'])
    

HTML(value='0 of 2 Examples annotated, Current Position: 0 ')

Output()

In [10]:
annotations

,example,changed,label
0,I love this movie,True,negative
1,I was really disappointed by the book,True,negative


In [8]:
pixt.annotate?


1.  query = _______
2.  build Glove vector of query
3.  sort documents by similarity
4.  Show a batch of the K most similar documents for labelling
5.  while not reach some metric
    1.  load/train/update classifier
    2.  rank unlabelled documents by uncertanty
    3. show a batch of K documents (with higher uncertanty)
    
